<a href="https://colab.research.google.com/github/ItsOpaz/KPI-AI-Generator/blob/main/KPI_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai
%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=65452ac5a63f5f4df284949a1adbf9f80909a321903e4fa830467c9084af339a
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
import os
import re
import openai
from google.colab import drive, auth
import gspread
from google.auth import default
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.master("local[*]").getOrCreate()

Data is held in google drive as google sheets, because google colab has been used to run this notebook.

In [ ]:
auth.authenticate_user()
creds, _ = default()
# for ease of use google sheets is used to handle data
gc = gspread.authorize(creds)
sh = gc.open('Toimialaratkaisut')

Below are functions used to call OpenAI API with various prompts
- changing prompts can result in better responses
- changing model can result in lower processing times
- max_tokens limits the lenght of response

In [ ]:
# OpenAI api key is in google drive
with open('/content/drive/My Drive/colab_data/openAi.txt', 'r') as f:
  API_KEY = f.readlines()[0]

def define_KPI(industry, segment):

  openai.api_key = API_KEY

  prompt = (f"list 5 most important {segment} KPI for company working in {industry} without explanations")
  response = openai.Completion.create(
    model="text-davinci-003", 
    prompt=prompt,
    max_tokens=60
  )
  return response["choices"][0]["text"]

def describe(KPI):

  openai.api_key = API_KEY

  prompt = (f"describe what does this {KPI} measure")

  response = openai.Completion.create(
    model="text-davinci-003", 
    prompt=prompt,
    max_tokens=120
  )

  return response["choices"][0]["text"]

def calculation_formula(KPI):

  openai.api_key = API_KEY

  prompt = (f"what is formula for {KPI}, output only formula")

  response = openai.Completion.create(
    model="text-davinci-003", 
    prompt=prompt,
    max_tokens=60
  )
  
  return response["choices"][0]["text"]

def default_value(KPI):

  openai.api_key = API_KEY

  prompt = (f"what is default value for {KPI}, output only value")

  response = openai.Completion.create(
    model="text-davinci-003", 
    prompt=prompt,
    max_tokens=60
  )

  return response["choices"][0]["text"]

def target_value(KPI):

  openai.api_key = API_KEY

  prompt = (f"what is target value for {KPI}, output only value")

  response = openai.Completion.create(
    model="text-davinci-003", 
    prompt=prompt,
    max_tokens=60
  )

  return response["choices"][0]["text"]

def data_source(KPI):

  openai.api_key = API_KEY

  prompt = (f"from which part of company’s information system is the data obtained for this {KPI}")
  response = openai.Completion.create(
    model="text-davinci-003", 
    prompt=prompt,
    max_tokens=60
  )

  return response["choices"][0]["text"]
  
# create spark user defined functions from these to be later used in withColumn()
# Spark is used to leverage multiprocessing
UDF_define = udf(lambda x, y: define_KPI(x, y), StringType())
UDF_describe = udf(lambda x: describe(x), StringType())
UDF_formula = udf(lambda x: calculation_formula(x), StringType())
UDF_default = udf(lambda x: default_value(x), StringType())
UDF_target = udf(lambda x: target_value(x), StringType())
UDF_source = udf(lambda x: data_source(x), StringType())

First step was splitting industry combinations to single industries to find out what industries are present in data and how big part of the data industry represents. Industries are filtered because we wanted to limit data which we are processing. Filtered industries are then saved to google sheets.

In [ ]:
data_worksheet = sh.worksheet("data")
rows = data_worksheet.get_all_values()
df = pd.DataFrame(rows)
df.columns = df.iloc[0]
df = df.iloc[1:]
df['Frequency'] = pd.to_numeric(df['Frequency'])

total_freq = sum(df['Frequency'])
sample_size = 12_000

df['CustomIndustry'] = df['CustomIndustry'].str.split(pat=',')
df = df.explode('CustomIndustry')
df = df.groupby('CustomIndustry').agg(Frequency=('Frequency', 'sum'), Count=('CustomIndustry', 'count'))
df = df.assign(Amount=lambda x: round(x.Frequency/total_freq * sample_size))
df = df[df['Amount'] >= 170].sort_values('Amount', ascending=False).reset_index()

industry_worksheet = sh.worksheet("Industries")
industry_worksheet.update([df.columns.values.tolist()] + df.values.tolist())

,Frequency,Count,Amount
CustomIndustry,,,
Wireless,33,5,11.0
Women's,5,1,2.0
Wood Processing,27,6,9.0
eSports,4,1,1.0
iOS,52,5,17.0


Lists of 5 KPI meters per industry per segment are created using OpenAI functions defined above. If there is need to change segments, only the segments list needs to be changed.

Adding new industries to data can be done by adding new rows to sheet "AI created KPIs" with industry as name

After KPI lists have been created, data is updated to google sheets.

In [ ]:
#segments to categorize KPIs
segments = ['strategic', 'operational', 'profitability', 'efficiency', 'customer']

Industry_worksheet = sh.worksheet("AI created KPIs")
rows = Industry_worksheet.get_all_values()
df = pd.DataFrame(rows)
df.columns = df.iloc[0]
df = df.iloc[1:]

# split dataframe to 2
processed_industries = df[df['strategic'] !='']
unprocessed_industries = df[df['strategic'] == '']

if not unprocessed_industries.empty:

  # convert pandas dataframe to spark for parallel processing
  data = spark.createDataFrame(unprocessed_industries)

  # create fields with KPIs from each category
  for s in segments:
    data = data.withColumn(str(s), UDF_define(data.Name, s))

  tmp = data.toPandas()

  # combine dataframes for worksheet update
  df = processed_industries.append(tmp, ignore_index=True)
  Industry_worksheet.update([df.columns.values.tolist()] + df.values.tolist())

KPIs are categorized by industry and segment for filtering purposes. Categorized data is updated to google sheets.

In [ ]:
Industry_worksheet = sh.worksheet("AI created KPIs")
rows = Industry_worksheet.get_all_values()
df = pd.DataFrame(rows)
df.columns = df.iloc[0]
df = df.iloc[1:]

# Split the contents of segment columns into separate rows, where each row represents a single item in a numbered list
for column in segments:
    split_df = df[column].str.split('\n').apply(pd.Series)
    split_df = split_df.rename(columns = lambda x : f"{column}_{x+1}")
    df = pd.concat([df.drop([column], axis=1), split_df], axis=1)

cols = [i for i in df.columns if i != "Name"]

# Unpivot columns to categorize KPIs using segments
df = df.melt(value_vars=cols, id_vars='Name', var_name='Type', value_name='KPI')

# Trim types to match each other
df['Type'] = df['Type'].apply(lambda x: x[:-2])

# Remove numbered list bullets
df['KPI'] = df['KPI'].apply(lambda x: x[2:])

# Trim white spaces and capitalize the first letter of each word in another column
df['KPI'] = df['KPI'].str.strip().str.title()

# Rename a column from "Name" to "Industry"
result = df.rename(columns={"Name": "Industry"})
result.head()

categorized_worksheet = sh.worksheet("Categorized KPI")
rows = categorized_worksheet.get_all_values()
tmp = pd.DataFrame(rows)
tmp.columns = tmp.iloc[0]
tmp = tmp.iloc[1:]

df = result.append(tmp, ignore_index=True)
df = df.drop_duplicates()
df.head()
categorized_worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '19sDfNpNox9LqG6Ae5DQ97WPppVA9R29Hu3sPoOz1Z08',
 'updatedRange': "'Categorized KPI'!A1:C1301",
 'updatedRows': 1301,
 'updatedColumns': 3,
 'updatedCells': 3903}

Distinct KPIs are filtered from the data. Notice that plurals wont be problem as we will be using fuzzy join in Power BI

In [ ]:
categorized_worksheet = sh.worksheet("Categorized KPI")
rows = categorized_worksheet.get_all_values()
df = pd.DataFrame(rows)
df.columns = df.iloc[0]
df = df.iloc[1:]

df = pd.DataFrame(df['KPI'])

# split possible abbrevation and ignore
df['KPI'] = df['KPI'].str.split(pat='(')
df['KPI'] = df['KPI'].apply(lambda x: x[0])
# remove special characters
df['KPI'] = df['KPI'].str.replace('\W\s', '', regex=True)
df['KPI'] = df['KPI'].str.strip().str.title()
# some abbrevations might be left so get rid of most of those
df = df[df['KPI'].apply(lambda x: len(str(x))) > 3]

df = df.drop_duplicates()

industry_worksheet = sh.worksheet("Distinct KPI")
industry_worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '19sDfNpNox9LqG6Ae5DQ97WPppVA9R29Hu3sPoOz1Z08',
 'updatedRange': "'Distinct KPI'!A1:A618",
 'updatedRows': 618,
 'updatedColumns': 1,
 'updatedCells': 618}

Additional information is created for unique KPIs using OpenAI. Adding new KPIs can be done by adding new KPI by name to new row in KPI sheet.

In [ ]:
KPI_worksheet = sh.worksheet("KPI")
rows = KPI_worksheet.get_all_values()
df = pd.DataFrame(rows)
df.columns = df.iloc[0]
df = df.iloc[1:]

# split dataframe to 2
defined_KPI = df[df['Description'] !='']
undefined_KPI = df[df['Description'] == '']

if not undefined_KPI.empty:
  
  # convert pandas dataframe to spark for parallel processing
  data = spark.createDataFrame(undefined_KPI)

  # add fields for dataframe
  data = data.withColumn("Description", UDF_describe(data.Name))
  data = data.withColumn("Calculation Formula", UDF_formula(data.Name))
  data = data.withColumn("Default Value", UDF_default(data.Name))
  data = data.withColumn("Target Value", UDF_target(data.Name))
  data = data.withColumn("Data Source", UDF_source(data.Name))

  tmp = data.toPandas()

  # combine dataframes for worksheet update
  df = defined_KPI.append(tmp, ignore_index=True)
  KPI_worksheet.update([df.columns.values.tolist()] + df.values.tolist())

,Name,Description,Calculation Formula,Default Value,Target Value,Data Source
626,Average Length of Stay,\n\nAverage Length of Stay measures the averag...,\n\nAVG LOS = Sum of Total Length of Stay / To...,\n\n2.8 days,\n\nThe target value for the Average Length of...,\n\nThe data for the Average Length of Stay me...
627,Acquisition Cost per User/Customer,\n\nAcquisition Cost per User/Customer measure...,\n\nAcquisition Cost per User/Customer = Total...,\n\n0,\n\nThis depends on the specific business goal...,\n\nThe Acquisition Cost per User/Customer met...
628,test,?\n\nThis test measures the understanding of b...,\n\nTest Measure = ( (# of Successes / # of At...,\n\n0,"\n\nIt depends on which test is being used, as...",\n\nIt is not possible to answer this question...
629,test2,\n\nThis test measures how well the user can d...,\n\nTest2 = (True Positives + True Negatives) ...,\n\n0.0,\n\n47,\n\nThe data for this test2 meter was obtained...
630,test3,\n\nTest3 measures the performance of a system...,\n\nTest3 Measure = (Test1 Measure + Test2 Mea...,\n\n0,\n\n60,\n\nThe data for this test3 meter was obtained...
